&#x1f12f; Javier Bejar - APA/GEI/FIB/UPC

In [ ]:
# Uncomment to upgrade packages
# !pip3 install pandas --user --upgrade --quiet
# !pip3 install numpy --user --upgrade --quiet
# !pip3 install scipy --user --upgrade --quiet
# !pip3 install statsmodels --user --upgrade --quiet
# !pip3 install seaborn --user --upgrade --quiet
# !pip3 install matplotlib --user --upgrade --quiet
# !pip3 install scikit-learn --user --upgrade 
# !pip install scikit-optimize --user --quiet
!pip install apafib --upgrade --user --quiet

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, HTML
show_html = lambda html: display(HTML(html))

from time import time
from datetime import timedelta

init_time = time()

# APA - Laboratorio - Sesión 5
## K-nearest neighbours - Multi Layer Perceptron

In [ ]:
import pandas as pd
from pandas import read_csv

import numpy as np
from numpy.random import choice
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import set_config

from sklearn.metrics import  ConfusionMatrixDisplay,\
                  classification_report,  RocCurveDisplay, PrecisionRecallDisplay,\
                    accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error

from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding

from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor


from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV,train_test_split, cross_val_score, TimeSeriesSplit

from yellowbrick.classifier.rocauc import roc_auc
from yellowbrick.target.feature_correlation import feature_correlation
from yellowbrick.classifier import precision_recall_curve

from apafib import load_electric_devices, load_energy

import warnings

set_config(display='text')
warnings.filterwarnings('ignore')
plt.rcParams.update({'font.size': 16})
# sns.set()
pd.set_option('display.precision', 3)

In [ ]:
def save_results(clf, X_test, y_test, nclf, df):
    df.loc[nclf,'test acc'] = accuracy_score(y_test, clf.predict(X_test))
    df.loc[nclf,'precision score (W)'] = precision_score(y_test, clf.predict(X_test), average='weighted')
    df.loc[nclf,'recall score (W)'] = recall_score(y_test, clf.predict(X_test), average='weighted')
    df.loc[nclf,'f1 score (W)'] = f1_score(y_test, clf.predict(X_test), average='weighted')
    return df

results_df = pd.DataFrame()

niter = 15
cv = 5

## Seccion 1: Electric Devices Consumption - Clasificación

Este conjunto de datos corresponde al patron de consumo de un dia (cada 15 minutos) de un conjunto de aparatos electrónicos comunes (7) en una casa. El objetivo es clasificar el patrón de consumo en el aparato correspondiente.

Este conjunto de datos solo tiene datos continuos, esto es más adecuado para estos dos modelos.


In [ ]:
data = load_electric_devices()
data.head()

In [ ]:
data.describe(include='all').T

Tenemos 7 clases con cierto imbalance entre ellas

In [ ]:
data['Class'].value_counts()

In [ ]:
cls = [str(v) for v in sorted(data['Class'].unique())]
cls

Podemos comprobar si tenemos datos perdidos

In [ ]:
for c in data.columns:
    if data[c].isna().sum()>0:
        print(c, data[c].isna().sum())

Separamos la clase de los atributos y obtenemos el conjunto de entrenamiento y test de manera estratificada

In [ ]:
X= data.iloc[:,1:]
y= data.loc[:,'Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

Es poco práctico el visualizar 96 atributos, podríamos comprobar las caracterísitcas de los atributos por ejemplo calculando test estadísticos sobre su distribución si queremos usar algún modelo que asuma alguna distribución a priori.


Podemos no obstante esperar cierta correlación entre instantes consecutivos

In [ ]:
corr = X_train.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.subplots(figsize=(10, 8))
sns.heatmap(corr, mask=mask, cmap='seismic',  center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5});


Para poder ver si hay alguna relación entre las variables y las clases podemos usar métodos de reducción de dimensionalidad, aplicaremos PCA en este caso. Primero estandarizaremos los datos

In [ ]:
sscaler = StandardScaler()
X_train_sd = sscaler.fit_transform(X_train)
X_test_sd = sscaler.transform(X_test)

In [ ]:
pca = PCA().fit(X_train_sd);

In [ ]:
fig = plt.figure(figsize=(8,6));
plt.plot(range(1,len(pca.explained_variance_ratio_ )+1),pca.explained_variance_ratio_ ,alpha=0.8,marker='.',label="Variancia Explicada");
y_label = plt.ylabel('Variancia explicada');
x_label = plt.xlabel('Componentes');
plt.plot(range(1,len(pca.explained_variance_ratio_ )+1),
         np.cumsum(pca.explained_variance_ratio_),
         c='red',marker='.',
         label="Variancia explicada acumulativa");
plt.legend();
plt.title('Porcentaje de variancia explicada por componente');

Podemos ver que la variancia de los datos esta distribuida por todos los componentes asi que una visualizacion en dos dimensiones nos va a dar una visión limitada.

In [ ]:
X_train_pca = pca.transform(X_train_sd)
X_test_pca = pca.transform(X_test_sd)
plt.figure(figsize=(8,8));
sns.scatterplot(x=X_train_pca[:,0], y=X_train_pca[:,1], hue=y_train, palette='tab10');

Aun así, podemos ver que hay cierta separabilidad entre las clases, aunque es dificil decir si todas son separables.

### K nearest neighbours

K-nn funciona a partir de la recuperación de los vecinos más cercanos a partir de su distancia, por lo que es importate que todos los atributos se encuenten en la misma escala asi que los normalizamos utilizando el MinMax scaler.

In [ ]:
scaler = MinMaxScaler()

X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

Debemos explorar el rango de hperparámetos de K-nn, basicamente cuantos vecinos usamos para hacer la predicción, que distancias usamos para recuperarlos, como combinamos las distancias y cual es la resolución del indice que nos permite recuperar los vecinos (kd-tree).

In [ ]:
knn =  KNeighborsClassifier()
print(np.mean(cross_val_score(knn,X_train_s,y_train,cv=10)))

In [ ]:
param = {'n_neighbors':[1, 3, 5, 7, 11, 15], 
          'weights':['distance', 'uniform'], 
          'leaf_size':[1, 5, 10, 20, 30],
          'metric': ['l2', 'l1', 'cosine']}

knn_gs =  GridSearchCV(knn,param,cv=cv, n_jobs=-1)
knn_gs.fit(X_train_s, y_train);

In [ ]:
show_html(pd.DataFrame(knn_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

Podemos ver que hay todo un conjuto de hiperparámetos que dan el mismo resutado (al menos al tercer decimal). Los resultados se ordenan lexicograficamente, por defecto la mejor solucion tendra un indice con un ejemplo por hoja. En un caso práctico, a igualdad de resultados, un indice con más ejemplos por hoja debería ser más rápido.

In [ ]:
print(classification_report(knn_gs.predict(X_test_s), y_test,target_names=cls))
results_df = save_results(knn_gs, X_test_s, y_test, 'KNN', results_df)

Podemos ver que el acierto en el test es consistente con la validación cruzada. No todas las clases tienen los mismos resultados, las ultimas dos son las que tienen peor resultado. Esto puede deberse básicamente a que no tienen una gran diferencia con el resto 

In [ ]:
plt.figure(figsize=(8,8));
ConfusionMatrixDisplay.from_estimator(knn_gs, X_test_s,y_test, display_labels=cls, ax=plt.subplot());

Aqui podemos ver que hay clases que tienen bastante confusión con las clases que peor resultado tienen.

In [ ]:
plt.figure(figsize=(8,8));
roc_auc(knn_gs, X_train_s, y_train, X_test_s, y_test, classes=cls);

Como vimos la ROC multiclase es más dificil de interpretar, pero podemos ver que hay una clase fácilmente separable del resto, el resto nos daran un relativamente alto porcentaje de falsos positivos (~20%) 

Podemos usar permutation importance para ver que atributos parecen más importantes para la clasificación. En este caso tenemos 96 atributos asi que reduciremos la muestra con la que se calcula.

In [ ]:
c = choice(X_test.shape[0], size=1000, replace=False)
pi = permutation_importance(knn_gs,X_test_s[c], y_test.to_numpy()[c], n_jobs=-1, random_state=0)
var_imp = pd.DataFrame({'importance': pi.importances_mean}, index=data.columns[1:])

Podemos ver que la importanci se distribuye entre bastantes horas, pero las 7:45 horas parece ser bastante significativa.

In [ ]:
var_imp.sort_values(by='importance').plot.barh(figsize=(10,20), legend=False);

Como vimos, K-nn tiene problemas cuando la dimensionalidad crece, en este caso podemos usar los componentes de PCA para reducir estas dimensiones y comprobar si podemos reducir el tamaño sin comprometer la calidad del resultado. 

En un caso práctico esto puede ayudarnos a reducir la memoria necesaria para almacenar el modelos (tenemos que guardar los datos originales para poder predecir)

In [ ]:
nc = 40
X_train_pca_s = scaler.fit_transform(X_train_pca[:,:nc])
X_test_pca_s = scaler.transform(X_test_pca[:,:nc])

In [ ]:
knn =  KNeighborsClassifier()
print(np.mean(cross_val_score(knn,X_train_pca_s,y_train,cv=10)))

In [ ]:
param = {'n_neighbors':[1, 3, 5, 7, 11, 15], 
          'weights':['distance', 'uniform'], 
          'leaf_size':[1, 5, 10, 20, 30],
          'metric': ['l2', 'l1', 'cosine']}

knn_gs =  GridSearchCV(knn,param,cv=cv, n_jobs=-1)
knn_gs.fit(X_train_pca_s, y_train);

In [ ]:
show_html(pd.DataFrame(knn_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

Podemos ver que reducir alrededor de un 40% los atributos no reduce demasiado el error de crosvalidacion, habría que ver si compensa en la práctica o tiene sentido en la aplicación real.

In [ ]:
print(classification_report(knn_gs.predict(X_test_pca_s), y_test,target_names=cls))
results_df = save_results(knn_gs, X_test_pca_s, y_test, 'KNN (PCA)', results_df)

El error en el conjunto de test es coherente, pero hemos perdido calidad en la mejor clase.

In [ ]:
plt.figure(figsize=(8,8));
ConfusionMatrixDisplay.from_estimator(knn_gs, X_test_pca_s,y_test, display_labels=cls, ax=plt.subplot());

In [ ]:
plt.figure(figsize=(8,8));
roc_auc(knn_gs, X_train_pca_s, y_train, X_test_pca_s, y_test, classes=cls);

Podemos ver en la curva ROC que hemos simplificado el espacio de decisión, las curvas son discretas.

## MLP

En el caso del MLP la forma en la que normalizamos los datos puede tener un impacto no solo en la calidad del resultado, sino tambien en la velocidad de convergencia. Usaremos el StandardScaler ya que converge más rápido, podeis cambiar vosotros la normalización para ver la diferencia.


In [ ]:
sdscaler = StandardScaler()

X_train_sd = sdscaler.fit_transform(X_train)
X_test_sd = sdscaler.transform(X_test)

El permitir early stopping reducira el tiempo de ajuste, es probable que el resultado sea algo peor a veces, pero reducimos la posibilidad de sobre ajuste.

In [ ]:
mlp = MLPClassifier(max_iter=10000, early_stopping=True, n_iter_no_change=15, random_state=0)
print(np.mean(cross_val_score(mlp,X_train_sd,y_train,cv=10)))

Los parametros por defecto dan un resultado parecido al Knn.

Haremos primero una exploración en cuadrícula probando algunos valores para los hiperparámetros.

In [ ]:
param = {'hidden_layer_sizes':[10, 50, 100, 200], 
         'activation':['relu', 'logistic', 'identity'], 
         'learning_rate_init': [0.001, 0.01, 0.1]  }

mlp =  MLPClassifier(max_iter=10000, early_stopping=True, n_iter_no_change=20,learning_rate='adaptive',random_state=0)
mlp_gs =  GridSearchCV(mlp,param,cv=cv, n_jobs=-1, refit=True)
mlp_gs.fit(X_train_sd, y_train);

In [ ]:
show_html(pd.DataFrame(mlp_gs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

Obtenemos un resultado algo peor. No podemos explorar completamente el espacio de hiperparámetros debido al coste de ajustar cada modelo.

Una alternativa es utilizar una exploración aleatoria o dirigida de alguna manera pero no explorando todas las posibilidades. Un método que permite una buena exploración es la búsqueda bayesians. En esta vamos muestreando el espacio de hiperparámetros y obteniendo una función substituto que aproxima la calidad del resultado para todo el espacio de valores de manera que podemos ir escogiendo combinaciones dependiendo de la calidad  que  estime esta función.

In [ ]:
from skopt import BayesSearchCV

In [ ]:
param = {'hidden_layer_sizes':[10, 50, 100, 200, 300], 
'activation':['relu', 'identity', 'logistic'], 
'alpha':[0.0001, 0.001, 0.01],
'momentum': [0.95, 0.90, 0.85, 0.8], 
'learning_rate_init': [0.001, 0.01, 0.1],
'n_iter_no_change':[10, 20, 40, 50], 
'learning_rate': ['constant', 'invscaling', 'adaptive']}

mlp =  MLPClassifier(max_iter=10000,early_stopping=True,random_state=0)
mlp_bs =  BayesSearchCV(mlp,param,
                        n_iter=niter, 
                        cv=cv, n_jobs=-1, 
                        refit=True,random_state=0)
mlp_bs.fit(X_train_sd, y_train);

In [ ]:
show_html(pd.DataFrame(mlp_bs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

Dado que hay aleatoriedad en la búsqueda el resultado puede variar entre ejecuciones, pero podemos limitar el numero de ajustes del modelo y explorarlo más.

In [ ]:
print(classification_report(mlp_gs.predict(X_test_sd), y_test,target_names=cls))
results_df = save_results(mlp_gs, X_test_sd, y_test, 'MLP', results_df)

El error del test es coherente con el de validacion cruzada. El resultado es algo diferente, hemos mejorado en alguna de las clases peores.

In [ ]:
plt.figure(figsize=(8,8));
ConfusionMatrixDisplay.from_estimator(mlp_bs, X_test_sd,y_test, display_labels=cls, ax=plt.subplot());

In [ ]:
plt.figure(figsize=(8,8));
roc_auc(mlp_bs, X_train_sd, y_train, X_test_sd, y_test, classes=cls);

El resultado de ese modelo es bastante parecido

In [ ]:
c = choice(X_test.shape[0], size=2000, replace=False)
pi = permutation_importance(mlp_gs,X_test_sd[c], y_test.to_numpy()[c], n_jobs=-1, random_state=0)
var_imp = pd.DataFrame({'importance': pi.importances_mean},
                       index=data.columns[1:])

La importancia de los atributos es diferente, pero el más importante es común entre los dos modelos

In [ ]:
var_imp.sort_values(by='importance').plot.barh(figsize=(10,20), legend=False);

In [ ]:
results_df.sort_values(by=['test acc'], ascending=False)

## Sección 2: Energy Data - Regresión

Un problema que se puede resolver mediante regresión es la predicción de series de tiempo.

La predicción de series temporales es un mundo en si mismo, pero también podemos usar modelos de aprendizaje para esta tarea. 

En este tipo de problemas queremos predecir valores de un momento en el tiempo a partir de los valores anteriores. En este caso debemos decidir cuantos instantes anteriores utilizamos y si utilizamos solo la variable objetivo o añadimos también otras variables que tengamos disponibles.

En este caso usaremos un conjunto de datos parecido al de clasificación, queremos predecir el consumo de energia de los electrodomesticos de una casa. Tenemos muchos otros atributos, pero en este caso solo utilizaremos la variable objetivo. Podéis encontrar la documentación de este conjunto de datos aqui (https://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction)


In [ ]:
data = load_energy()

niter = 15
cv = 5

In [ ]:
data.head()

In [ ]:
data.describe(include='all')

Seleccionamos la variable que queremos predecir.

In [ ]:
energy = data.loc[:,'Appliances']

In [ ]:
energy.describe()

Seleccionamos un conjunto de entrenamiento (los 12000 primeros ejemplos) y de test (el resto).

En este tipo de problemas no podemos hacer la partición del conjunto de datos como en el resto de problemas que hemos tratado. Fijaos que los datos no son iid. Ademas si partieramos aleatoriamente el conjunto de datos estariamos mezclando instantes en el tiempo, lo que nos interesa es poder ajustar el modelo con los datos del pasado y predecir los futuros.

In [ ]:
e_train, e_test =  energy.iloc[:12000], energy.iloc[12000:]

In [ ]:
e_train.shape, e_test.shape

Para generar el conjunto de datos debemos generar una matriz de datos en la que tengamos ventanas de la serie.

Para ello tenemos una función de numpy que tiene ese proposito.

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view

Seleccionamos un tamaño de ventana para hacer la matriz de datos. Usamos un tamaño w+1, asi tenemos la ventana anterior al dato a predecir en las primeras w posiciones y el dato a predecir en la última. 

La función que genera las ventanas nos retornará una matriz 3D, donde una de las dimensiones es una sola columna, la función `squeeze` nos elimina la columna redundante.

Generamos la matriz de datos para el conjunto de entrenamiento y el de test.

Normalizaremos primero los datos. Para Knn no deberia importar ya que todos los atributos se encuentran en el mismo rango, para MLP puede ayudar la convergencia ya que los datos estan en un rango de valores muy grande,

In [ ]:
w = 4

sdscaler = MinMaxScaler()

e_train_s = sdscaler.fit_transform(e_train.to_numpy().reshape(-1, 1))
e_test_s = sdscaler.transform(e_test.to_numpy().reshape(-1, 1))

windows_train = sliding_window_view(e_train_s, w+1, axis=0).copy()
X_train_w, y_train_w = windows_train.squeeze()[:,:-1], windows_train.squeeze()[:,-1]

windows_test = sliding_window_view(e_test_s, w+1, axis=0).copy()
X_test_w, y_test_w = windows_test.squeeze()[:,:-1], windows_test.squeeze()[:,-1]

In [ ]:
X_train_w.shape, X_test_w.shape, y_train_w.shape, y_test_w.shape

### K nearest neighbours

El escalado de los datos en este caso sería innecesario ya que estamos utilizando la misma variable para generar todas las ventanas, asi que todas estarán un mismo rango. En todo caso normalizaríamos la variable antes de generar las ventanas.

En este caso usamos Knn para regresión y exploramos el rango de hiper parámetros. 

Igual que no podemos mezclar el tiempo en los datos de entrenamiento y test, la validacion cruzada tampoco se puede realizar de la misma manera, nos hemos de asegurar de que la particion de entrenamiento siempre este en el pasado y la de validacion en el futuro y que no compartan ventanas de datos entre ellas. Esto nos lo permite TimeSeriesSplit. La medida que usamos para elegir el modelo es MSE.

In [ ]:
pd.set_option('display.precision', 5)
knn =  KNeighborsRegressor()

In [ ]:
param = {'n_neighbors':[1, 3, 5, 7, 11, 15, 20, 25], 
         'weights':['distance', 'uniform'], 
         'leaf_size':[1, 5, 10, 15, 20, 25, 30],
         'metric': ['l2', 'l1', 'cosine']}

knn_bs = BayesSearchCV(knn,param,n_iter=niter, 
                        cv=TimeSeriesSplit(n_splits=cv, gap=w+1), 
                        scoring=make_scorer(mean_squared_error, greater_is_better=False),
                        n_jobs=-1, 
                        refit=True, random_state=0)         
knn_bs.fit(X_train_w, y_train_w);

In [ ]:
show_html(pd.DataFrame(knn_bs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

El error de MSE no esta en las unidades de los datos, calcular la raiz cuadrada del error nos pondrá en esas unidades.

In [ ]:
mean_squared_error(y_test_w,knn_bs.predict(X_test_w)), mean_absolute_error(y_test_w,knn_bs.predict(X_test_w))

El error en el test es parecido al de validacion cruzada (incluso algo mejor).

Podemos visualizar la predicción de los primeros datos del conjunto de test para ver como de cerca esta la prediccion del modelo de la realidad. No es un metodo de validacion, pero al menos nos permitirá ver si realmente estamos prediciendo algo.

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(y_test_w[:500],'r');
plt.plot(knn_bs.predict(X_test_w[:500,:]),'b');

Podemos ver que la predicción sigue hasta cierto punto el principio de la serie del conjunto de test.

### MLP

Ahora usamos el MLP para regresión.

Exploramos los diferentes hiperparámetros del MLP, en este caso hacemos directamente una búsqueda bayesiana.

In [ ]:
param = {'hidden_layer_sizes':[100, 200, 300], 
         'activation':['relu',  'logistic'], 
         'alpha':[0.0001, 0.001, 0.01],
         'momentum': [0.95, 0.90, 0.85], 
         'learning_rate_init': [0.001, 0.01, 0.1],
         'n_iter_no_change':[30, 40, 50], 
         'learning_rate': ['constant', 'invscaling', 'adaptive']}

mlp =  MLPRegressor(max_iter=10000,early_stopping=True,random_state=0)
mlp_bs = BayesSearchCV(mlp,param,n_iter=niter, 
                        cv=TimeSeriesSplit(n_splits=cv, gap=w+1), 
                        scoring=make_scorer(mean_squared_error, greater_is_better=False),
                        n_jobs=-1, 
                        refit=True, random_state=0)    
mlp_bs.fit(X_train_w, y_train_w);

In [ ]:
show_html(pd.DataFrame(mlp_bs.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score').head().to_html())

In [ ]:
mean_squared_error(y_test_w,mlp_bs.predict(X_test_w)), mean_absolute_error(y_test_w,mlp_bs.predict(X_test_w))

El MSE es coherente con el error de validación cruzada y es posiblemente ligeramente mejor que el de KNN, pero cada vez que ajustemos el MLP obtendremos un error diferente.

Podemos tambien superponer las predicciones del test sobre el valor real.

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(y_test_w[:500],'r');
plt.plot(mlp_bs.predict(X_test_w[:500,:]),'b');

Deberíamos explorar diferentes longitudes de ventana para ver cual es la que obtiene el mejor error, en este tipo de problemas este valor es otro hiperparámetro.

In [ ]:
print(f"Total Running time {timedelta(seconds=(time() - init_time))}")